In [1]:
import pandas as pd
import json

In [2]:
apartments = pd.read_csv('../data/curated/geometry_property.csv')

In [6]:
apartments[1:13]

,Address,Suburb,Prices,Estimated resident population (no.),Population density (persons/km2),Median age - persons (years),Working age population (aged 15-64 years) (no.),Employed (no.),Unemployed (no.),Renter (no.),...,Value of residential building ($m),Houses - total (no.),Townhouses - total (no.),Apartments - total (no.),Median weekly household rental payment ($),Rented (no.),Count of homeless persons (no.),"Used at least one form of public transport (train, tram, bus, ferry) (no.)",latitude,longitude
1,"7/1 Mabel Street, \nIVANHOE VIC 3079",IVANHOE,450.0,12561.0,2487.4,40.9,8357.0,0.0,0.0,27.0,...,35.0,2911.0,1189.0,1529.0,440.0,1505.0,16.0,343.0,-37.761896,145.029437
2,"2/59 Green Street, \nIVANHOE VIC 3079",IVANHOE,450.0,12561.0,2487.4,40.9,8357.0,0.0,0.0,27.0,...,35.0,2911.0,1189.0,1529.0,440.0,1505.0,16.0,343.0,-37.764732,145.035655
3,"104/15 Ivanhoe Parade, \nIVANHOE VIC 3079",IVANHOE,450.0,12561.0,2487.4,40.9,8357.0,0.0,0.0,27.0,...,35.0,2911.0,1189.0,1529.0,440.0,1505.0,16.0,343.0,-37.767871,145.044723
4,"63 Dunne Street, \nKINGSBURY VIC 3083",KINGSBURY,450.0,10564.0,1172.6,34.0,7808.0,0.0,0.0,53.0,...,8.0,2381.0,831.0,1250.0,369.0,1285.0,120.0,382.0,-37.714056,145.037364
5,"29 Tandarra Crescent, \nLALOR VIC 3075",LALOR,450.0,23663.0,4579.1,75.1,15020.0,0.0,0.0,121.0,...,42.0,7707.0,515.0,572.0,716.0,2182.0,154.0,648.0,-37.664019,145.013961
6,"12 Julie Court, \nLANGWARRIN VIC 3910",LANGWARRIN,450.0,25248.0,671.1,38.4,16450.0,0.0,0.0,64.0,...,47.0,8037.0,1431.0,25.0,365.0,1530.0,49.0,151.0,-38.143911,145.208457
7,"8/18 Johnstone Street, \nMALVERN VIC 3144",MALVERN,450.0,22221.0,2936.7,38.2,14931.0,0.0,0.0,25.0,...,105.0,4789.0,2224.0,2731.0,421.0,2540.0,205.0,746.0,-37.867502,145.035036
8,"22 Campbell Street, \nFRANKSTON VIC 3199",FRANKSTON,2100.0,62521.0,4843.5,121.2,39777.0,0.0,0.0,455.0,...,68.0,21147.0,4743.0,1416.0,1096.0,8000.0,650.0,751.0,-38.144601,145.143083
9,"1584 Boneo Road, \nFLINDERS VIC 3929",FLINDERS,2150.0,6021.0,21.0,56.2,3295.0,0.0,0.0,3.0,...,79.0,3864.0,25.0,30.0,424.0,246.0,13.0,12.0,-38.459304,144.903582
10,"84 Toorak Road West, \nSOUTH YARRA VIC 3141",SOUTH YARRA,2200.0,28123.0,21662.2,106.5,22280.0,0.0,0.0,76.0,...,79.0,1181.0,2094.0,15905.0,1278.0,8671.0,64.0,1947.0,-37.837510,144.979980


In [8]:
tram_stops = pd.read_csv('../data/curated/tram_stop.csv')
train_stations = pd.read_csv('../data/curated/train_station.csv')
schools = pd.read_csv('../data/curated/school_locations.csv')
restaurants = pd.read_csv('../data/curated/melb_resto.csv')
landmarks = pd.read_csv('../data/curated/melb_landmarks.csv')

In [10]:
# Calculate Haversine Distance
import math

def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Radius of the earth in km
    dLat = math.radians(lat2 - lat1)
    dLon = math.radians(lon2 - lon1)
    a = math.sin(dLat / 2) * math.sin(dLat / 2) + math.cos(math.radians(lat1)) \
        * math.cos(math.radians(lat2)) * math.sin(dLon / 2) * math.sin(dLon / 2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    d = R * c  # Distance in km
    return d

In [11]:
for index, row in apartments.iterrows():
    lon1 = row['longitude']
    lat1 = row['latitude']

    num_stops = 0 
    for index_stops, row2 in tram_stops.iterrows():
        lon2 = row2['LONGITUDE']
        lat2 = row2['LATITUDE']

        distance = haversine(lat1, lon1, lat2, lon2)
        if distance <= 0.5:
            num_stops += 1

    apartments.at[index, 'num_stops'] = num_stops

In [12]:
for index, row in apartments.iterrows():
    lon1 = row['longitude']
    lat1 = row['latitude']
    
    num_stations = 0
    for index_stations, row3 in train_stations.iterrows():
        lon2 = row3['LONGITUDE']
        lat2 = row3['LATITUDE']
    
        distance = haversine(lat1, lon1, lat2, lon2)
        if distance <= 0.5:
            num_stations += 1
    apartments.at[index, 'num_stations'] = num_stations

In [13]:
for index, row in apartments.iterrows():
    lon1 = row['longitude']
    lat1 = row['latitude']
    
    num_schools = 0
    for index_schools, row4 in schools.iterrows():
        lon2 = row4['X']
        lat2 = row4['Y']
    
        distance = haversine(lat1, lon1, lat2, lon2)
        if distance <= 2:
            num_schools += 1
    apartments.at[index, 'num_schools'] = num_schools

In [14]:
for index, row in apartments.iterrows():
    lon1 = row['longitude']
    lat1 = row['latitude']
    
    num_restaurants = 0
    for index_restaurants, row5 in restaurants.iterrows():
        lon2 = row5['longitude']
        lat2 = row5['latitude']
    
        distance = haversine(lat1, lon1, lat2, lon2)
        if distance <= 1:
            num_restaurants += 1
    apartments.at[index, 'num_restaurants'] = num_restaurants

In [15]:
# Preprocess the 'co_ordinates' column to ensure it's valid JSON
landmarks['co_ordinates'] = landmarks['co_ordinates'].str.replace("'", "\"")
landmarks['co_ordinates'] = landmarks['co_ordinates'].apply(json.loads)

landmarks['lon'] = landmarks['co_ordinates'].apply(lambda x:x['lon'])
landmarks['lat'] = landmarks['co_ordinates'].apply(lambda x:x['lat'])

In [16]:
for index, row in apartments.iterrows():
    lon1 = row['longitude']
    lat1 = row['latitude']
    
    num_landmarks = 0
    for index_landmarks, row6 in landmarks.iterrows():
        lon2 = row6['lon']
        lat2 = row6['lat']
    
        distance = haversine(lat1, lon1, lat2, lon2)
        if distance <= 1:
            num_landmarks += 1
    apartments.at[index, 'num_landmarks'] = num_landmarks

In [22]:
apartments['total_transport'] = apartments['num_stops'] + apartments['num_stations']
apartments.sort_values(by='num_restaurants', ascending=False)

,Address,Suburb,Prices,Estimated resident population (no.),Population density (persons/km2),Median age - persons (years),Working age population (aged 15-64 years) (no.),Employed (no.),Unemployed (no.),Renter (no.),...,Count of homeless persons (no.),"Used at least one form of public transport (train, tram, bus, ferry) (no.)",latitude,longitude,num_stops,num_stations,num_schools,num_restaurants,num_landmarks,total_transport
4215,"3909/1 Queensbridge Square, \nSOUTHBANK VIC 3006",SOUTHBANK,725.0,16538.0,20569.7,30.9,14882.0,0.0,0.0,48.0,...,30.0,1844.0,-37.821212,144.961919,18.0,0.0,10.0,1127.0,66.0,18.0
909,"1109/7 Riverside Quay, \nSOUTHBANK VIC 3006",SOUTHBANK,620.0,16538.0,20569.7,30.9,14882.0,0.0,0.0,48.0,...,30.0,1844.0,-37.821256,144.964540,19.0,1.0,10.0,1081.0,62.0,20.0
1267,"3101/7 Riverside Quay, \nSOUTHBANK VIC 3006",SOUTHBANK,1449.0,16538.0,20569.7,30.9,14882.0,0.0,0.0,48.0,...,30.0,1844.0,-37.821256,144.964540,19.0,1.0,10.0,1081.0,62.0,20.0
621,"7801/7 Riverside Quay, \nSOUTHBANK VIC 3006",SOUTHBANK,1800.0,16538.0,20569.7,30.9,14882.0,0.0,0.0,48.0,...,30.0,1844.0,-37.821256,144.964540,19.0,1.0,10.0,1081.0,62.0,20.0
2000,"41/1 Riverside Quay, \nSOUTHBANK VIC 3006",SOUTHBANK,650.0,16538.0,20569.7,30.9,14882.0,0.0,0.0,48.0,...,30.0,1844.0,-37.821256,144.964540,19.0,1.0,10.0,1081.0,62.0,20.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1564,"51/1 Graham Street, \nPORT MELBOURNE VIC 3207",PORT MELBOURNE,520.0,16373.0,5869.7,43.2,11248.0,0.0,0.0,46.0,...,44.0,379.0,-37.841867,144.943494,0.0,0.0,1.0,0.0,0.0,0.0
1565,"202/503 Plenty Road, \nPRESTON VIC 3072",PRESTON,520.0,33825.0,6306.2,75.0,24317.0,0.0,0.0,241.0,...,273.0,1434.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
1566,"2/100 Madeline Street, \nPRESTON VIC 3072",PRESTON,520.0,33825.0,6306.2,75.0,24317.0,0.0,0.0,241.0,...,273.0,1434.0,-37.733168,145.018538,5.0,0.0,11.0,0.0,0.0,5.0
1567,"4/977-979 High Street, \nRESERVOIR VIC 3073",RESERVOIR,520.0,52065.0,10943.5,154.1,35092.0,0.0,0.0,490.0,...,416.0,1900.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
# Define the absolute path for saving the file
file_path = '../data/curated/places_property.csv'

# Save the DataFrame to the specified absolute path
apartments.to_csv(file_path, index=False)